In [4]:
import numpy as np
import pandas as pd
import keras.backend as K
from keras import optimizers
from keras.layers import Conv1D, SpatialDropout1D
from keras.layers import Activation, Lambda, concatenate
from keras.layers import Convolution1D, Dense, Flatten
from keras.models import Input, Model
import keras.layers

## Generating the data

In [5]:
def data_generator(n, seq_length):
    """
    Args:
        seq_length: Length of the adding problem data
        n: # of data in the set
    """
    x_num = np.random.uniform(0, 1, (n, 1, seq_length))
    x_mask = np.zeros([n, 1, seq_length])
    y = np.zeros([n, 1])
    for i in range(n):
        positions = np.random.choice(seq_length, size=2, replace=False)
        x_mask[i, 0, positions[0]] = 1
        x_mask[i, 0, positions[1]] = 1
        y[i, 0] = x_num[i, 0, positions[0]] + x_num[i, 0, positions[1]]
    x = np.concatenate((x_num, x_mask), axis=1)
    x = np.transpose(x, (0, 2, 1))
    return x, y

In [6]:
x_train, y_train = data_generator(n=50000, seq_length=100)
x_test, y_test = data_generator(n=1000, seq_length=100)

In [7]:
def channel_normalization(x):
    # Normalize by the highest activation
    max_values = K.max(K.abs(x), 2, keepdims=True) + 1e-5
    out = x / max_values
    return out


def temporal_block(x, s, i, activation, nb_filters, kernel_size):
    original_x = x
    conv = Conv1D(filters=nb_filters, kernel_size=kernel_size,
                  dilation_rate=i, padding='causal',
                  name='dilated_conv_%d_tanh_s%d' % (i, s))(x)
    if activation == 'norm_relu':
        x = Activation('relu')(conv)
        x = Lambda(channel_normalization)(x)

    x = SpatialDropout1D(0.05)(x)

    # 1x1 conv.
    x = Convolution1D(nb_filters, 1, padding='same')(x)
    res_x = keras.layers.add([original_x, x])
    return res_x, x


def tcn(num_feat, num_classes, nb_filters,
                kernel_size, dilations,max_len,
                activation='norm_relu', use_skip_connections=True,
                output_slice_index=None,
                regression=False):
    """
    dilation_depth : number of layers per stack
    nb_stacks : number of stacks.
    """
    input_layer = Input(name='input_layer', shape=(max_len, num_feat))
    x = input_layer
    x = Convolution1D(nb_filters, kernel_size, padding='causal', name='initial_conv')(x)

    skip_connections = []
    for s, i in enumerate(dilations):
            x, skip_out = temporal_block(x, s, i, activation, nb_filters, kernel_size)
            skip_connections.append(skip_out)

    if use_skip_connections:
        x = keras.layers.add(skip_connections)
    x = Activation('relu')(x)
    
    # Downsample to desired number of output sequences
    x = Lambda(lambda tt: tt[:, output_slice_index, :])(x)
    print('x.shape=', x.shape)

    x = Dense(num_classes)(x)
    x = Activation('linear', name='output_dense')(x)
    output_layer = x
    model = Model(input_layer, output_layer)
    adam = optimizers.Adam(lr=0.002, clipnorm=1.)
    model.compile(adam, loss='mean_squared_error')

    return model


In [8]:
model = tcn(output_slice_index=-1,
                       num_feat=x_train.shape[2],
                       num_classes=1,
                       nb_filters=24,
                       kernel_size=7,
                       dilations=[2**i for i in range(6)],
                       max_len=x_train.shape[1],
                       activation='norm_relu',
                       use_skip_connections=False,
                       regression=True)
model.summary()

x.shape= (?, 24)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 100, 2)       0                                            
__________________________________________________________________________________________________
initial_conv (Conv1D)           (None, 100, 24)      360         input_layer[0][0]                
__________________________________________________________________________________________________
dilated_conv_1_tanh_s0 (Conv1D) (None, 100, 24)      4056        initial_conv[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 100, 24)      0           dilated_conv_1_tanh_s0[0][0]     
____________________________________________________________________________________________

In [9]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=256)

Train on 50000 samples, validate on 1000 samples
Epoch 1/5
50000/50000 [==============================] - 7s 147us/step - loss: 0.1928 - val_loss: 0.1579
Epoch 2/5
50000/50000 [==============================] - 4s 77us/step - loss: 0.0893 - val_loss: 0.0153
Epoch 3/5
50000/50000 [==============================] - 4s 77us/step - loss: 0.0221 - val_loss: 0.0064
Epoch 4/5
50000/50000 [==============================] - 4s 77us/step - loss: 0.0129 - val_loss: 0.0046
Epoch 5/5
50000/50000 [==============================] - 4s 77us/step - loss: 0.0101 - val_loss: 0.0035


In [11]:
x_test[10:11]

array([[[0.55167333, 0.        ],
        [0.35246029, 0.        ],
        [0.16584834, 0.        ],
        [0.33941679, 0.        ],
        [0.71252437, 0.        ],
        [0.27922037, 0.        ],
        [0.75522695, 0.        ],
        [0.12811373, 0.        ],
        [0.50160507, 0.        ],
        [0.27709846, 0.        ],
        [0.31475355, 0.        ],
        [0.70910291, 0.        ],
        [0.04072749, 1.        ],
        [0.3880072 , 0.        ],
        [0.5919275 , 0.        ],
        [0.14629439, 0.        ],
        [0.43198356, 0.        ],
        [0.75850719, 0.        ],
        [0.66504123, 0.        ],
        [0.78722347, 0.        ],
        [0.85609317, 0.        ],
        [0.11388875, 0.        ],
        [0.12272511, 0.        ],
        [0.01525723, 0.        ],
        [0.9407971 , 0.        ],
        [0.19421055, 0.        ],
        [0.46345692, 0.        ],
        [0.09522939, 0.        ],
        [0.73178341, 0.        ],
        [0.051

In [12]:
pred = model.predict(x_test[10:11])
pred

array([[0.04831929]], dtype=float32)

In [13]:
0.04072749+0.00132871

0.042056199999999995